In [ ]:
import tensorflow as tf
import os
tf.random.set_seed(42)
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, Input


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/

'Colab Notebooks'   dataset


In [ ]:
# Defining Path
DATASET_PATH = "/content/drive/MyDrive/dataset"

In [ ]:
# Define image size and batch size
IMAGE_SIZE = (224,224) # Resize all images to 224x224 pixel values
BATCH_SIZE = 32        # Number of images per batch

In [ ]:
# Load dataset from directory
dataset = image_dataset_from_directory(
    DATASET_PATH,
    labels='inferred', # Infer labels from folder names
    label_mode="int",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 9786 files belonging to 4 classes.


In [ ]:
# def modify_labels(image, label):
#     class_label = tf.cast(label, tf.float32)  # Classification label
#     reg_label = tf.cast(label, tf.float32)    # Regression label (same as class)
#     return image, (class_label, reg_label)

# # Apply transformation
# train_dataset = train_dataset.map(modify_labels)
# val_dataset = val_dataset.map(modify_labels)

In [ ]:
# Split the dataset into training, validation, and test sets
train_size = 0.7
val_size = 0.2
test_size = 0.1

In [ ]:
# Getting the number of batches
total_batches = tf.data.experimental.cardinality(dataset).numpy()
train_batches = int(total_batches * train_size)
val_batches = int(total_batches * val_size)
print(total_batches,train_batches,val_batches)

306 214 61


In [ ]:
# Split the data
train_dataset = dataset.take(train_batches)
remaining = dataset.skip(train_batches)
val_dataset = remaining.take(val_batches)
test_dataset = remaining.skip(val_batches)


In [ ]:
for image, label in train_dataset.take(1):
    print(f"Image shape: {image.shape}, Label: {label.numpy()}")

Image shape: (32, 224, 224, 3), Label: [0 0 2 0 3 0 1 1 0 2 2 2 1 2 2 2 2 3 0 2 2 3 2 2 2 2 1 2 0 2 1 2]


In [ ]:
print(f"Label shape: {label.shape}, Unique labels: {set(label.numpy())}")

Label shape: (32,), Unique labels: {0, 1, 2, 3}


In [ ]:
# Normalize the pizel values to [0,1]
def normalize(image,label):
    image = tf.cast(image,tf.float32) / 255.0
    return image,label

In [ ]:
# Prefetch for performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
import tensorflow.keras.backend as K

# Custom function to restrict output between 0 and 4
def bounded_relu(x):
    return K.clip(x, 0, 4)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models

In [ ]:
# Define the input shape
INPUT_SHAPE = (224,224,3) # Image size with 3 color channels (RGB)

# Creating the Convolution Neural Network model

# Load EfficientNetB0 as feature extractor
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False  # Freezing the pretrained layers

def build_model():
    inputs = layers.Input(shape=INPUT_SHAPE)

    # Convolution layers for feature extraction
    # inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)  # Convert feature maps into a single vector
    x = Dropout(0.3)(x)  # Regularization

    # The neural network built by me
    x = layers.Conv2D(32,(3,3), activation="relu",padding="same")(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(64,(3,3), activation="relu",padding="same")(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(128,(3,3), activation="relu",padding="same")(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128,activation="relu")(x)

    # Classification Head (Yes/No Arthritis)
    # class_output = layers.Dense(1,activation='sigmoid',name='classification')(x)
    class_output = layers.Dense(5, activation='softmax', name='classification')(x)

    # Regression Head (Severity Grade 0-4)
    # reg_output = layers.Dense(1,activation='linear',name='regression')(x)
    # Update the regression output layer
    reg_output = layers.Dense(1, activation=bounded_relu, name='regression')(x)

    # Define the model tow outputs
    model = models.Model(inputs=inputs,outputs=[class_output,reg_output])

    return model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten
import tensorflow.keras.backend as K

# Define a custom bounded ReLU activation function
def bounded_relu(x):
    return K.clip(x, 0, 4)

# Define the input shape
INPUT_SHAPE = (224, 224, 3)

# Load EfficientNetB0 as feature extractor
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
base_model.trainable = False  # Freeze pretrained layers

def build_model():
    inputs = Input(shape=INPUT_SHAPE)

    # Extract features using EfficientNetB0 (output shape: (None, h, w, channels))
    x = base_model(inputs, training=False)
    x = Dropout(0.3)(x)  # Regularization

    # Additional convolutional layers (these expect a 4D tensor)
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    # x = MaxPooling2D((2, 2))(x)

    # Instead of GlobalAveragePooling2D, use Flatten to preserve spatial details
    x = Flatten()(x)

    # Further processing with a Dense layer
    x = Dense(128, activation="relu")(x)

    # Classification Head: 5 classes using softmax
    # class_output = Dense(5, activation='softmax', name='classification')(x)
    # Instead of name='classification', use:
    class_output = Dense(5, activation='softmax', name='classification_output')(x)

    # Regression Head: Use bounded ReLU to restrict output between 0 and 4
    # reg_output = Dense(1, activation=bounded_relu, name='regression')(x)
    reg_output = Dense(1, activation=bounded_relu, name='bounded_regression_output')(x)

    # Define the model with two outputs
    # model = models.Model(inputs=inputs, outputs=[class_output, reg_output])
    # return model

    # Append Lambda layers to force the outputs to the correct 2D shape.
    # For classification, we want shape (batch_size, 5)
    class_output = layers.Lambda(lambda t: tf.reshape(t, (-1, 5)))(class_output)
    # For regression, we want shape (batch_size, 1)
    reg_output = layers.Lambda(lambda t: tf.reshape(t, (-1, 1)))(reg_output)

    # Define the model with these reshaped outputs.
    model = models.Model(inputs=inputs, outputs=[class_output, reg_output])

    # Build the model
    model = build_model()
    return model
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ efficientnetb0            │ (None, 7, 7, 1280)     │      4,049,571 │ input_layer_2[0][0]    │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 7, 7, 1280)     │              0 │ efficientnetb0[1][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 7, 7, 32)       │        368,672 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 3, 3, 32)       │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 3, 3, 64)       │         18,496 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 1, 1, 64)       │              0 │ conv2d_4[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 1, 1, 128)      │         73,856 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 128)            │              0 │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 128)            │         16,512 │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ classification_output     │ (None, 5)              │            645 │ dense_1[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bounded_regression_output │ (None, 1)              │            129 │ dense_1[0][0]          │
│ (Dense)                   │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,527,881 (17.27 MB)

 Trainable params: 478,310 (1.82 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
import tensorflow.keras.backend as K

# Custom activation function to clip output
def bounded_relu(x):
    return K.clip(x, 0, 4)  # Restricts values between 0 and 4

In [ ]:
# Building and compiling the model
model = build_model()

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
#  Compiling with two Loss functions
# model.compile(
#     optimizer=Adam(learning_rate=0.0001),
#     loss={'classification':'binary_crossentropy','regression':'mse'},
#     metrics={'classification':'accuracy','regression':'mae'}
# )

In [ ]:
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
#     loss={
#          'classification': 'sparse_categorical_crossentropy',
#          'regression': tf.keras.losses.Huber()  # Using the Huber loss object
#     },
#     metrics={
#          'classification': 'accuracy',
#          'regression': 'mae'
#     }
# )

# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),  # Lowered LR to avoid NaN
#     loss={
#         'classification': 'sparse_categorical_crossentropy',
#         'regression': 'mae'
#     },
#     metrics={
#         'classification': 'accuracy',
#         'regression': 'mae'
#     }
# )
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss={'classification_output': 'sparse_categorical_crossentropy', 'bounded_regression_output': tf.keras.losses.Huber()},
    metrics={'classification_output': 'accuracy', 'bounded_regression_output': 'mae'}
)


In [ ]:
# Print the model summary
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten, Lambda
import tensorflow.keras.backend as K

# Define a custom bounded ReLU activation function
def bounded_relu(x):
    return K.clip(x, 0, 4)

# Define the input shape
INPUT_SHAPE = (224, 224, 3)

# Load EfficientNetB0 as feature extractor
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
base_model.trainable = False  # Freeze pretrained layers

def build_model():
    inputs = Input(shape=INPUT_SHAPE)

    # Extract features using EfficientNetB0
    x = base_model(inputs, training=False)
    x = Dropout(0.3)(x)  # Regularization

    # Additional convolutional layers
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    # Do not use an additional pooling layer to avoid negative dimensions

    # Instead of GlobalAveragePooling2D, use Flatten to preserve spatial details
    x = Flatten()(x)

    # Further processing with a Dense layer
    x = Dense(128, activation="relu")(x)

    # Classification Head: 5 classes using softmax
    class_output = Dense(5, activation='softmax', name='classification_output')(x)

    # Regression Head: Use bounded ReLU to restrict output between 0 and 4
    reg_output = Dense(1, activation=bounded_relu, name='bounded_regression_output')(x)

    # Append Lambda layers to ensure proper output shape
    class_output = Lambda(lambda t: tf.reshape(t, (-1, 5)))(class_output)
    reg_output = Lambda(lambda t: tf.reshape(t, (-1, 1)))(reg_output)

    # Define the final model with two outputs
    model = models.Model(inputs=inputs, outputs=[class_output, reg_output])
    return model

# Build and summarize the model
model = build_model()
model.summary()


Model: "functional_931"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_935           │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ efficientnetb0            │ (None, 7, 7, 1280)     │      4,049,571 │ input_layer_935[0][0]  │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_932 (Dropout)     │ (None, 7, 7, 1280)     │              0 │ efficientnetb0[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2794 (Conv2D)      │ (None, 7, 7, 32)       │        368,672 │ dropout_932[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1862        │ (None, 3, 3, 32)       │              0 │ conv2d_2794[0][0]      │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2795 (Conv2D)      │ (None, 3, 3, 64)       │         18,496 │ max_pooling2d_1862[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1863        │ (None, 1, 1, 64)       │              0 │ conv2d_2795[0][0]      │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2796 (Conv2D)      │ (None, 1, 1, 128)      │         73,856 │ max_pooling2d_1863[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_931 (Flatten)     │ (None, 128)            │              0 │ conv2d_2796[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_931 (Dense)         │ (None, 128)            │         16,512 │ flatten_931[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ classification_output     │ (None, 5)              │            645 │ dense_931[0][0]        │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bounded_regression_output │ (None, 1)              │            129 │ dense_931[0][0]        │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_1862 (Lambda)      │ (None, 5)              │              0 │ classification_output… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_1863 (Lambda)      │ (None, 1)              │              0 │ bounded_regression_ou… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,527,881 (17.27 MB)

 Trainable params: 478,310 (1.82 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
# Number of training epochs
EPOCHS = 20 # Starting with 10 (Later change accordingly during hyperr parmeter tunning)
CHECKPOINT_PATH = "best_model.h5"

# Callback to save the best model (based on validation loss)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    CHECKPOINT_PATH,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [ ]:
def modify_labels(image, label):
  class_label = tf.cast(label, tf.float32)  # Classification label
  reg_label = tf.cast(label, tf.float32)    # Regression label (same as class)
  return image, (class_label, reg_label)  # Return a tuple of labels

# Apply function to dataset
train_dataset = train_dataset.map(modify_labels)
val_dataset = val_dataset.map(modify_labels)
test_dataset = test_dataset.map(modify_labels)
for image, (class_label, reg_label) in train_dataset.take(1):

  print(f"Classification Label: {class_label.numpy()}")
  print(f"Regression Label: {reg_label.numpy()}")

Classification Label: [[[[[2. 1. 2. 2. 0. 3. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 1.
     2. 2. 3. 2. 3. 2. 0. 0. 2. 1.]
    [2. 1. 2. 2. 0. 3. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 1.
     2. 2. 3. 2. 3. 2. 0. 0. 2. 1.]]

   [[2. 1. 2. 2. 0. 3. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 1.
     2. 2. 3. 2. 3. 2. 0. 0. 2. 1.]
    [2. 1. 2. 2. 0. 3. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 1.
     2. 2. 3. 2. 3. 2. 0. 0. 2. 1.]]]


  [[[2. 1. 2. 2. 0. 3. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 1.
     2. 2. 3. 2. 3. 2. 0. 0. 2. 1.]
    [2. 1. 2. 2. 0. 3. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 1.
     2. 2. 3. 2. 3. 2. 0. 0. 2. 1.]]

   [[2. 1. 2. 2. 0. 3. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 1.
     2. 2. 3. 2. 3. 2. 0. 0. 2. 1.]
    [2. 1. 2. 2. 0. 3. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 1.
     2. 2. 3. 2. 3. 2. 0. 0. 2. 1.]]]]



 [[[[2. 1. 2. 2. 0. 3. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 2. 1.
     2. 2. 3. 2. 3. 2. 0. 0. 2. 1.]
 

In [ ]:
import tensorflow.keras.backend as K
K.clear_session()

In [ ]:
# Training the model

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]
)

Epoch 1/20


ValueError: Expected keys ListWrapper(['lambda_1862', 'lambda_1863']) in loss dict, but found loss.keys()=['classification_output', 'bounded_regression_output']

In [ ]:
model.evaluate(test_dataset)

In [ ]:
# model.save('best_59_percent.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.application import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,Dropout, GlobalAveragePooling2D,Input

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten, Reshape
import tensorflow.keras.backend as K

# Define a custom bounded ReLU activation function
def bounded_relu(x):
    return K.clip(x, 0, 4)

# Define the input shape
INPUT_SHAPE = (224, 224, 3)

# Load EfficientNetB0 as feature extractor (without top layers)
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
base_model.trainable = False  # Freeze pretrained layers

# Function to build the model
def build_model():
    inputs = Input(shape=INPUT_SHAPE)

    # Feature extraction using EfficientNetB0
    x = base_model(inputs, training=False)
    x = Dropout(0.3)(x)  # Regularization

    # Additional convolutional layers
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)

    # Flatten the output instead of using GlobalAveragePooling
    x = Flatten()(x)

    # Fully connected layer
    x = Dense(128, activation="relu")(x)

    # Classification output (5 classes, softmax activation)
    class_output = Dense(5, activation='softmax', name='classification_output')(x)
    class_output = Reshape((5,), name="classification_reshape")(class_output)  # ✅ Using Reshape

    # Regression output (bounded ReLU activation for range 0 to 4)
    reg_output = Dense(1, activation=bounded_relu, name='bounded_regression_output')(x)
    reg_output = Reshape((1,), name="bounded_regression_reshape")(reg_output)  # ✅ Using Reshape

    # Define the model
    model = models.Model(inputs=inputs, outputs=[class_output, reg_output])
    return model

# Build the model
model = build_model()

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss={
        "classification_reshape": "sparse_categorical_crossentropy",
        "bounded_regression_reshape": "mean_squared_error"
    },
    metrics={
        "classification_reshape": ["accuracy"],
        "bounded_regression_reshape": ["mae"]
    }
)

# Display the model architecture
model.summary()

# Load your dataset (assuming train_dataset and val_dataset are already prepared)
# Training the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5
)
